### comment out the code below to install required packages

In [1]:
# code to install all the packages we need
# %pip install numpy
# %pip install pandas
# %pip install yfinance
# %pip install requests
# %pip install beautifulsoup4
# %pip install scikit-learn
# %pip install TA-lib
# %pip install joblib
# %pip install pickle-mixin
# %pip install Riskfolio-lib


In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import yfinance as yf
from ibapi.client import *
from ibapi.wrapper import *
from ibapi.contract import *
from ibapi.order import *
import ibapi

import requests
import bs4 as bs
import datetime
import time
import threading
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV  
import joblib

import talib as ta
from talib import MA_Type
import pickle


In [2]:
init_event = threading.Event() # init event
id_event = threading.Event()
mkt_event = threading.Event()
hist_event = threading.Event()
order_event = threading.Event()
evec_event = threading.Event()
port_event = threading.Event()
value_event = threading.Event()

In [3]:
# define the App class
class App(EClient, EWrapper):
    def __init__(self, address, port, cid):
        EClient.__init__(self, self)
        # # list to store data
        self.bar_dict = {}
        self.mkt_price = []
        # create a connection with IBKR
        self.connect(address, port, cid)
        self.last_portfolio = pd.DataFrame(columns=["ticker","quantity","marketPrice","marketValue","averageCost","unrealizedPNL","realizedPNL"])
        self.value = 0
        # start client
        thread = threading.Thread(target=self.run)
        thread.start()
        init_event.set()


    def nextValidId(self, orderId: int):
        # provide a new order id for each of my requests
        super().nextValidId(orderId)
        logging.debug("setting nextValidOrderId: %d", orderId)
        self.nextValidOrderId = orderId
        print("NextValidId:", orderId)
        id_event.set()

    def tickPrice(self, reqId: int, tickType: int, price: float, attrib: ibapi.common.TickAttrib):
            print("Tick Price. Ticker Id:", reqId, "tickType:", tickType, "Price:", price)
            if tickType == 9: # if tickType is Close Price
                self.mkt_price.append([reqId, price])
                mkt_event.set()

    def historicalData(self, reqId, bar):
        if reqId not in self.bar_dict.keys():
            self.bar_dict[reqId] = []
        self.bar_dict[reqId].append(vars(bar))
        
    def historicalDataEnd(self, reqId, start, end):
        print(f"end of historicalData")
        hist_event.set()


    # implement code to monitor trade status and receive confirmation of the trade
    def openOrder(self, orderId: OrderId, contract: Contract, order: Order, orderstate: OrderState):
        # openorder callback
        print(f"openOrder. orderId:{orderId}, contract:{contract}, order:{order}")
        order_event.set()

    def orderStatus(self, orderId: OrderId, status: str, filled: float, reamining: float, avgFillPrice: float,
                    permId: int, parenId: int, lastFillPrice: float, clientId: int, whyHeld:str, mktCapPrice: float):
                    # orderstatus callback
        print(f"orderStatus. orderId: {orderId}, status: {status}, filled: {filled}, remaining:{reamining}, avgFillPrice: {avgFillPrice}, permId:{permId}, parentId:{parenId}, lastFillPrice: {lastFillPrice}, clientId: {clientId}, whyHeld: {whyHeld}, mktCapPrice:{mktCapPrice}")

    def execDetails(self, reqId: int, contract: Contract, execution: Execution):
        print(f"execDetails. reqId: {reqId}, contract: {contract}, execution: {execution}")
        evec_event.set()
    # basically a summary

    def commissionReport(self, commissionReport: CommissionReport):
        super().commissionReport(commissionReport)
        print("CommissionReport.", commissionReport)
        
    # called when query portfolio information
    def updatePortfolio(self, contract, position, marketPrice, marketValue, averageCost, unrealizedPNL, realizedPNL, accountName):
        self.last_portfolio = pd.concat([self.last_portfolio,
                                        pd.DataFrame([[contract.symbol, position, marketPrice, marketValue, averageCost, unrealizedPNL, realizedPNL]],
                                                    columns=self.last_portfolio.columns)],
                                        ignore_index=True)
        port_event.set()

    # query total value
    def accountSummary(self, reqId: int, account: str, tag: str, value: str, currency: str):
        self.value = float(value)
        print("Total value of the account: ", self.value)
    
    def accountSummaryEnd(self, reqId: int):
        print('end of account summary')
        value_event.set()

In [ ]:
# Connect to the TWS API
app = App('127.0.0.1', 7497, 1000)
init_event.wait() # wait until it's connected
init_event.clear()

ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:eufarmnj
ERROR:ibapi.wrapper:ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR:ibapi.wrapper:ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR:ibapi.wrapper:ERROR -1 2106 HMDS data farm connection is OK:ushmds.nj
ERROR:ibapi.wrapper:ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR:ibapi.wrapper:ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR:ibapi.wrapper:ERROR -1 2158 Sec-def data farm connection is OK:secdefil


NextValidId: 138


In [5]:
app.reqMarketDataType(1) # in case it doesn't work, change 1 to 3

In [6]:
with open('portfolio_list.pickle', 'rb') as f:
    portfolio_list = pickle.load(f)

print(portfolio_list)

['BAC', 'CCL', 'DISH', 'FITB', 'FRC', 'F', 'NUE', 'VFC', 'VTRS', 'ZION']


# query the current portfolio


In [7]:
app.reqAccountUpdates(True, "DU6730183") #TODO: replace the DU222526 with our account ID
# port_event.wait()
# port_event.clear()

time.sleep(10)
last_portfolio = app.last_portfolio
app.reqAccountUpdates(False, "DU6730183") 

ERROR:ibapi.wrapper:ERROR -1 2100 API client has been unsubscribed from account data.


In [8]:
last_portfolio

,ticker,quantity,marketPrice,marketValue,averageCost,unrealizedPNL,realizedPNL
0,CCL,8169,9.826000,80268.60,9.785000,334.93,0.0
1,DISH,9252,8.506001,78697.52,8.925000,-3876.58,0.0
2,FIS,1469,56.613773,83165.63,55.175000,2113.56,0.0
3,FLT,380,217.306824,82576.59,208.275000,3432.09,0.0
4,FRC,5872,13.586362,79779.12,13.995000,-2399.52,0.0
5,LNC,3747,20.746103,77735.65,20.962966,-812.59,0.0
6,MET,1345,59.486000,80008.67,59.065000,566.25,0.0
7,USB,2247,35.243900,79193.04,35.565000,-721.51,0.0
8,VTRS,8053,9.956000,80175.67,9.895000,491.24,0.0
9,WBA,2184,35.574001,77693.62,35.825000,-548.18,0.0


In [9]:
app.reqAccountUpdates(False, "DU6730183")#TODO: replace the DU222526 with our account ID
app.reqAccountSummary(9001, "All", "NetLiquidation")
value_event.wait()
value_event.clear()
# time.sleep(3)
total_value = app.value
app.cancelAccountSummary(9001)

Total value of the account:  830853.73
end of account summary


In [10]:
total_value

830853.73

# implementation
* decide what to buy & sell, as well as the quantity

In [11]:

# get the market price
for i, t in enumerate(portfolio_list):
    contract = Contract()
    contract.symbol = t
    contract.secType = "STK"
    contract.currency = "USD"
    contract.exchange = "SMART"
    app.reqMktData(i, contract, "", False, False, [])
    mkt_event.wait()
    mkt_event.clear()
    app.cancelMktData(i)

mkt_df = pd.DataFrame(app.mkt_price, columns=['reqId', 'mkt_price'])
assert sum(mkt_df.isnull().any()) == 0, "didn't get all the market price"
assert len(mkt_df) == len(portfolio_list), "didn't get all the market price"


Tick Price. Ticker Id: 0 tickType: 4 Price: 28.56
Tick Price. Ticker Id: 0 tickType: 6 Price: 28.66
Tick Price. Ticker Id: 0 tickType: 7 Price: 28.24
Tick Price. Ticker Id: 0 tickType: 9 Price: 28.48
Tick Price. Ticker Id: 0 tickType: 14 Price: 28.59
Tick Price. Ticker Id: 0 tickType: 1 Price: 28.56
Tick Price. Ticker Id: 0 tickType: 2 Price: 28.57
Tick Price. Ticker Id: 1 tickType: 1 Price: 9.79
Tick Price. Ticker Id: 1 tickType: 2 Price: 9.8
Tick Price. Ticker Id: 1 tickType: 4 Price: 9.8
Tick Price. Ticker Id: 1 tickType: 6 Price: 9.85
Tick Price. Ticker Id: 1 tickType: 7 Price: 9.65
Tick Price. Ticker Id: 1 tickType: 9 Price: 9.62
Tick Price. Ticker Id: 1 tickType: 14 Price: 9.65


ERROR:ibapi.wrapper:ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.hfarm
ERROR:ibapi.wrapper:ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.hfarm
ERROR:ibapi.wrapper:ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.jfarm
ERROR:ibapi.wrapper:ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.jfarm
ERROR:ibapi.wrapper:ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.eufarmnj
ERROR:ibapi.wrapper:ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.eufarmnj


Tick Price. Ticker Id: 2 tickType: 1 Price: 8.48
Tick Price. Ticker Id: 2 tickType: 2 Price: 8.49
Tick Price. Ticker Id: 2 tickType: 4 Price: 8.49
Tick Price. Ticker Id: 2 tickType: 6 Price: 8.55
Tick Price. Ticker Id: 2 tickType: 7 Price: 8.06
Tick Price. Ticker Id: 2 tickType: 9 Price: 8.08
Tick Price. Ticker Id: 2 tickType: 14 Price: 8.11
Tick Price. Ticker Id: 3 tickType: 4 Price: 26.5
Tick Price. Ticker Id: 3 tickType: 6 Price: 26.71
Tick Price. Ticker Id: 3 tickType: 7 Price: 26.02
Tick Price. Ticker Id: 3 tickType: 9 Price: 26.39
Tick Price. Ticker Id: 3 tickType: 14 Price: 26.35
Tick Price. Ticker Id: 3 tickType: 1 Price: 26.48
Tick Price. Ticker Id: 3 tickType: 2 Price: 26.49
Tick Price. Ticker Id: 4 tickType: 1 Price: 13.53
Tick Price. Ticker Id: 4 tickType: 2 Price: 13.54
Tick Price. Ticker Id: 4 tickType: 4 Price: 13.54
Tick Price. Ticker Id: 4 tickType: 6 Price: 13.94
Tick Price. Ticker Id: 4 tickType: 7 Price: 13.53
Tick Price. Ticker Id: 4 tickType: 9 Price: 13.82
Tick P

In [12]:
target_portfolio = pd.DataFrame(columns=['tickers', 'weight', 'mkt_price'])
target_portfolio['tickers'] = portfolio_list
target_portfolio['weight'] = np.repeat(1/len(portfolio_list), len(portfolio_list))
target_portfolio['mkt_price'] = mkt_df['mkt_price']

In [13]:
target_portfolio['quantity'] = np.floor(target_portfolio['weight'] * (total_value * 0.95) / target_portfolio['mkt_price'])

In [14]:
last_portfolio

,ticker,quantity,marketPrice,marketValue,averageCost,unrealizedPNL,realizedPNL
0,CCL,8169,9.826000,80268.60,9.785000,334.93,0.0
1,DISH,9252,8.506001,78697.52,8.925000,-3876.58,0.0
2,FIS,1469,56.613773,83165.63,55.175000,2113.56,0.0
3,FLT,380,217.306824,82576.59,208.275000,3432.09,0.0
4,FRC,5872,13.586362,79779.12,13.995000,-2399.52,0.0
5,LNC,3747,20.746103,77735.65,20.962966,-812.59,0.0
6,MET,1345,59.486000,80008.67,59.065000,566.25,0.0
7,USB,2247,35.243900,79193.04,35.565000,-721.51,0.0
8,VTRS,8053,9.956000,80175.67,9.895000,491.24,0.0
9,WBA,2184,35.574001,77693.62,35.825000,-548.18,0.0


In [15]:
target_portfolio

,tickers,weight,mkt_price,quantity
0,BAC,0.1,28.48,2771.0
1,CCL,0.1,9.62,8204.0
2,DISH,0.1,8.08,9768.0
3,FITB,0.1,26.39,2990.0
4,FRC,0.1,13.82,5711.0
5,F,0.1,12.67,6229.0
6,NUE,0.1,147.66,534.0
7,VFC,0.1,21.57,3659.0
8,VTRS,0.1,9.82,8037.0
9,ZION,0.1,30.29,2605.0


ERROR:ibapi.wrapper:ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.usfuture
ERROR:ibapi.wrapper:ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.usfuture


# figure out the best order of our order. (To save commission fee)

* target_portfolio
> target_portfolio python pandas dataframe containing information of stocks portfolio(target portfolio) that I want to form through trading. The first column of the dataframe is stocks' tickers and the second column of the dataframe is the quantity of each stock that I want to hold. 

* last_portfolio
> last_portfolio is a dataframe containing information of last portfolio that I hold

In [16]:
# Create a list to store the stocks that need to be sold
stocks_to_sell = []
# Create a list to store the stocks that need to buy
stocks_to_buy = []

# Iterate through the target_portfolio dataframe
for index, row in target_portfolio.iterrows():
    # Check if the stock is present in the last_portfolio dataframe
    last_portfolio_stock = last_portfolio[last_portfolio['ticker'] == row['tickers']]
    if not last_portfolio_stock.empty:
        # Compare the quantities
        last_portfolio_quantity = float(last_portfolio_stock['quantity'].values[0])
        if row['quantity'] < last_portfolio_quantity:
            # Calculate the difference in quantity
            quantity_diff = last_portfolio_quantity - row['quantity'] 
            # Add the stock to the list of stocks to sell
            stocks_to_sell.append([row['tickers'], quantity_diff])

        if row['quantity'] > last_portfolio_quantity:
            # Calculate the difference in quantity
            quantity_diff = row['quantity'] - last_portfolio_quantity 
            # Add the stock to the list of stocks to buy
            stocks_to_buy.append([row['tickers'], quantity_diff])

# we also need to sell the stocks that are in last_portfolio but not in target_portfolio
# Find the difference between the two portfolios
new_tickers_to_sell = set(last_portfolio['ticker']).difference(set(target_portfolio['tickers']))

for ticker in new_tickers_to_sell:
    # Find the quantity of the stock to sell from the last_portfolio
    quantity = last_portfolio.loc[last_portfolio['ticker'] == ticker, 'quantity'].values[0]
    # append to stocks to sell
    if quantity != 0:
        stocks_to_sell.append([ticker, quantity])

# we also need to buy the stocks that are in target_protfolio but not in last_portfolio
# Find the difference between the two portfolios
new_tickers_to_buy = set(target_portfolio['tickers']).difference(set(last_portfolio['ticker']))

for ticker in new_tickers_to_buy:
    # Find the quantity of the stock to buy from the target_portfolio
    quantity = target_portfolio.loc[target_portfolio['tickers'] == ticker, 'quantity'].values[0]
    # append to stocks to buy
    if quantity != 0:
        stocks_to_buy.append([ticker, quantity])

In [17]:
stocks_to_buy

[['CCL', 35.0],
 ['DISH', 516.0],
 ['F', 6229.0],
 ['NUE', 534.0],
 ['VFC', 3659.0],
 ['FITB', 2990.0],
 ['ZION', 2605.0],
 ['BAC', 2771.0]]

In [18]:
stocks_to_sell

[['FRC', 161.0],
 ['VTRS', 16.0],
 ['USB', Decimal('2247')],
 ['WBA', Decimal('2184')],
 ['MET', Decimal('1345')],
 ['FLT', Decimal('380')],
 ['LNC', Decimal('3747')],
 ['FIS', Decimal('1469')]]

# place order

In [19]:
app.reqIds(-1) # require a new id
id_event.wait()
id_event.clear()
orderid = app.nextValidOrderId
for stock in stocks_to_sell:
    # Replace the place holder values with the appropriate values
    ticker = stock[0]
    quantity = stock[1]
    contract = Contract()
    contract.symbol = ticker
    contract.secType = "STK"
    contract.exchange = "SMART"
    contract.currency = "USD"
    order = Order()
    order.action = "SELL"
    order.orderType = "MKT"
    order.totalQuantity = quantity
    order.tif = "GTC" # default is day order

    orderid += 1

    app.placeOrder(orderid, contract, order)
    order_event.wait()
    order_event.clear()

count = 0
for stock in stocks_to_buy:
    # Replace the place holder values with the appropriate values
    app.reqAccountUpdates(False, "DU6730183")#TODO: replace the DU222526 with our account ID
    app.reqAccountSummary(9001, "All", "SettledCash")
    value_event.wait()
    value_event.clear()
    cash_value = app.value
    if count >= len(stocks_to_buy)/2 and cash_value <= 0:
        break # stop buying when cash is below 0
    ticker = stock[0]
    quantity = stock[1]
    contract = Contract()
    contract.symbol = ticker
    contract.secType = "STK"
    contract.exchange = "SMART"
    contract.currency = "USD"
    order = Order()
    order.action = "BUY"
    order.orderType = "MKT"
    order.totalQuantity = quantity
    order.tif = "GTC" # default is day order

    orderid += 1
    app.placeOrder(orderid, contract, order)
    order_event.wait()
    order_event.clear()
    count +=1

NextValidId: 138
openOrder. orderId:139, contract:81731135,FRC,STK,,0,?,,SMART,,USD,FRC,FRC,False,,,,combo:, order:139,1000,2102925340: MKT SELL 161@13.54 GTC
orderStatus. orderId: 139, status: PreSubmitted, filled: 0, remaining:161, avgFillPrice: 0.0, permId:2102925340, parentId:0, lastFillPrice: 0.0, clientId: 1000, whyHeld: , mktCapPrice:0.0
execDetails. reqId: -1, contract: 81731135,FRC,STK,,0,,,ISLAND,,USD,FRC,FRC,False,,,,combo:, execution: ExecId: 00025b44.6437fac8.01.01, Time: 20230413 10:58:55 US/Eastern, Account: DU6730183, Exchange: ISLAND, Side: SLD, Shares: 161, Price: 13.54, PermId: 2102925340, ClientId: 1000, OrderId: 139, Liquidation: 0, CumQty: 161, AvgPrice: 13.54, OrderRef: , EvRule: , EvMultiplier: 0, ModelCode: , LastLiquidity: 2
openOrder. orderId:139, contract:81731135,FRC,STK,,0,?,,SMART,,USD,FRC,FRC,False,,,,combo:, order:139,1000,2102925340: MKT SELL 161@13.54 GTC
orderStatus. orderId: 139, status: Filled, filled: 161, remaining:0, avgFillPrice: 13.54, permId:

execDetails. reqId: -1, contract: 276329,ZION,STK,,0,,,MEMX,,USD,ZION,NMS,False,,,,combo:, execution: ExecId: 00025b47.64378d8d.01.01, Time: 20230413 10:58:58 US/Eastern, Account: DU6730183, Exchange: MEMX, Side: BOT, Shares: 2, Price: 30.69, PermId: 2102925354, ClientId: 1000, OrderId: 153, Liquidation: 0, CumQty: 114, AvgPrice: 30.69, OrderRef: , EvRule: , EvMultiplier: 0, ModelCode: , LastLiquidity: 2
openOrder. orderId:153, contract:276329,ZION,STK,,0,?,,SMART,,USD,ZION,NMS,False,,,,combo:, order:153,1000,2102925354: MKT BUY 2605@0 GTC
orderStatus. orderId: 153, status: Submitted, filled: 114, remaining:2491, avgFillPrice: 30.69, permId:2102925354, parentId:0, lastFillPrice: 30.69, clientId: 1000, whyHeld: , mktCapPrice:0.0
openOrder. orderId:153, contract:276329,ZION,STK,,0,?,,SMART,,USD,ZION,NMS,False,,,,combo:, order:153,1000,2102925354: MKT BUY 2605@0 GTC
orderStatus. orderId: 153, status: Submitted, filled: 114, remaining:2491, avgFillPrice: 30.69, permId:2102925354, parentId:

In [20]:
app.disconnect()